In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Nov 28 12:01:27 PST 2018
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.5 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.866
BogoMIPS:              5851.93
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


#### Make directories

In [2]:
%%bash
mkdir /home/sam/data/geoduck
mkdir /home/sam/data/geoduck/transcriptomes
mkdir /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits

#### Transfer Transdecoder coding sequences FastA file

In [3]:
%%bash

cd /home/sam/data/geoduck/transcriptomes

# Uncomment following line to retrieve file using wget
# wget http://gannet.fish.washington.edu/Atumefaciens/20181121_geo_transdecoder/20180827_trinity_geoduck.fasta.transdecoder.cds
rsync gannet:/volume1/web/Atumefaciens/20181121_geo_transdecoder/20180827_trinity_geoduck.fasta.transdecoder.cds
ls -lh

-rw-r--r--    283,967,118 2018/11/23 15:42:16 20180827_trinity_geoduck.fasta.transdecoder.cds
total 4.0K
drwxrwxr-x 2 sam sam 4.0K Nov 28 12:04 transdecoder_fasta_splits


#### Split mulit-FastA file in to individual FastA files with PyFaidx

In [4]:
%%bash
cd /home/sam/data/geoduck/transcriptomes/

# Count sequences in FastA
echo "-------------------"
echo "NUMBER OF SEQUENCES IN ORIGINAL FASTA"
grep -c ">" 20180827_trinity_geoduck.fasta.transdecoder.cds
echo "-------------------"
echo ""
echo ""

cd /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/

# Split FastA
time \
/home/sam/software/bin/pyfaidx-0.5.5.2 \
--split-files \
../20180827_trinity_geoduck.fasta.transdecoder.cds

# Count number of individual FastA files
echo "-------------------"
echo "NUMBER OF INDIVIDUAL FASTA FILES"
ls -1 | wc -l
echo "-------------------"

-------------------
NUMBER OF SEQUENCES IN ORIGINAL FASTA
-------------------


-------------------
NUMBER OF INDIVIDUAL FASTA FILES
0
-------------------


grep: 20180827_trinity_geoduck.fasta.transdecoder.cds: No such file or directory
Traceback (most recent call last):
  File "/home/sam/software/bin/pyfaidx-0.5.5.2", line 9, in <module>
    load_entry_point('pyfaidx==0.5.5.2', 'console_scripts', 'faidx')()
  File "build/bdist.linux-x86_64/egg/pyfaidx/cli.py", line 197, in main
  File "build/bdist.linux-x86_64/egg/pyfaidx/cli.py", line 21, in write_sequence
  File "build/bdist.linux-x86_64/egg/pyfaidx/__init__.py", line 996, in __init__
  File "build/bdist.linux-x86_64/egg/pyfaidx/__init__.py", line 368, in __init__
pyfaidx.FastaNotFoundError: Cannot read FASTA file ../20180827_trinity_geoduck.fasta.transdecoder.cds

real	0m0.150s
user	0m0.104s
sys	0m0.020s


Whoops! Ran ```rsync``` command incorrectly. Duh!

#### Transfer Transdecoder coding sequences FastA file (again)

In [5]:
%%bash
cd /home/sam/data/geoduck/transcriptomes
rsync \
--archive \
gannet:/volume1/web/Atumefaciens/20181121_geo_transdecoder/20180827_trinity_geoduck.fasta.transdecoder.cds .

echo "-------------------"

ls -lh

-------------------
total 271M
-rw-r--r-- 1 sam users 271M Nov 23 15:42 20180827_trinity_geoduck.fasta.transdecoder.cds
drwxrwxr-x 2 sam sam   4.0K Nov 28 12:04 transdecoder_fasta_splits


#### Split mulit-FastA file in to individual FastA files with PyFaidx

In [6]:
%%bash
cd /home/sam/data/geoduck/transcriptomes/

# Count sequences in FastA
echo "-------------------"
echo "NUMBER OF SEQUENCES IN ORIGINAL FASTA"
grep -c ">" 20180827_trinity_geoduck.fasta.transdecoder.cds
echo "-------------------"
echo ""
echo ""

cd /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/

# Split FastA
time \
/home/sam/software/bin/pyfaidx-0.5.5.2 \
--split-files \
../20180827_trinity_geoduck.fasta.transdecoder.cds

# Count number of individual FastA files
echo "-------------------"
echo "NUMBER OF INDIVIDUAL FASTA FILES"
ls -1 | wc -l
echo "-------------------"

-------------------
NUMBER OF SEQUENCES IN ORIGINAL FASTA
210586
-------------------


-------------------
NUMBER OF INDIVIDUAL FASTA FILES
210586
-------------------



real	0m43.789s
user	0m36.340s
sys	0m6.656s


#### Find complete coding sequences that contain typical terms for vitellogenin

##### The code below does the following:

- uses ```grep``` to search for vitellogenin terms and the word "complete"

- translates commas to spaces to aid in parsing/formatting for sort command

- sorts in reverse order, using the "version" flag to help sort strings that contain numbers and sorts on column 7 (i.e. the "score" column

- awk prints a header line to provide column descriptions and then prints out desired columns

- ```column -t``` formats output into nicely spaced columns

- ```tee``` prints output to file and to screen

In [7]:
%%bash
cd /home/sam/data/geoduck/transcriptomes/
grep --ignore-case "Vitellogenin" 20180827_trinity_geoduck.fasta.transdecoder.cds \
| grep "complete" \
| tr "," " " \
| sort -Vr -k7 \
| awk 'BEGIN{print "transcript_ID", "transcript_type", "transcript_length", "strand", "score", "annotation"}; \
{print $1, $4, $5, $6, $7 , $8}' \
| column -t \
| tee 20181127_geoduck_Vitellogenin_cds_matches.txt

transcript_ID                   transcript_type  transcript_length  strand  score          annotation
>TRINITY_DN51983_c0_g1_i8.p1    type:complete    len:4828           (-)     score=1005.69  sp|Q9U943|APLP_LOCMI|27.089|3.28e-109
>TRINITY_DN51983_c0_g1_i4.p1    type:complete    len:4680           (-)     score=971.32   sp|Q9U943|APLP_LOCMI|27.089|2.82e-109
>TRINITY_DN3302_c0_g1_i4.p1     type:complete    len:4277           (+)     score=941.33   sp|Q9U943|APLP_LOCMI|29.133|9.20e-117
>TRINITY_DN3302_c0_g1_i8.p1     type:complete    len:4277           (+)     score=941.04   sp|Q9U943|APLP_LOCMI|29.133|1.08e-116
>TRINITY_DN3302_c0_g1_i1.p1     type:complete    len:4277           (+)     score=941.04   sp|Q9U943|APLP_LOCMI|29.133|1.08e-116
>TRINITY_DN1469_c0_g4_i1.p1     type:complete    len:6053           (-)     score=859.49   sp|Q2PZL6|FAT4_MOUSE|38.356|1.43e-08
>TRINITY_DN1469_c0_g3_i1.p1     type:complete    len:6053           (+)     score=859.49   sp|Q2PZL6|FAT4_MOUSE|38.356|1.43e-

In [8]:
%%bash
cd /home/sam/data/geoduck/transcriptomes/
grep --ignore-case "Vtg" 20180827_trinity_geoduck.fasta.transdecoder.cds \
| grep "complete" \
| tr "," " " \
| sort -Vr -k7 \
| awk 'BEGIN{print "transcript_ID", "transcript_type", "transcript_length", "strand", "score", "annotation"}; \
{print $1, $4, $5, $6, $7 , $8}' \
| column -t \
| tee 20181127_geoduck_Vtg_cds_matches.txt

transcript_ID  transcript_type  transcript_length  strand  score  annotation


In [9]:
%%bash
cd /home/sam/data/geoduck/transcriptomes/
grep --ignore-case "Vg" 20180827_trinity_geoduck.fasta.transdecoder.cds \
| grep "complete" \
| tr "," " " \
| sort -Vr -k7 \
| awk 'BEGIN{print "transcript_ID", "transcript_type", "transcript_length", "strand", "score", "annotation"}; \
{print $1, $4, $5, $6, $7 , $8}' \
| column -t \
| tee 20181127_geoduck_Vg_cds_matches.txt

transcript_ID                   transcript_type  transcript_length  strand  score         annotation
>TRINITY_DN306895_c2_g2_i2.p1   type:complete    len:106            (-)     score=-21.67  zf-LITAF-like|PF10601.8|2.4e+03
>TRINITY_DN306895_c2_g2_i1.p1   type:complete    len:106            (-)     score=-21.67  zf-LITAF-like|PF10601.8|2.4e+03
>TRINITY_DN48461_c1_g1_i1.p1    type:complete    len:119            (-)     score=-4.86   VGCC_alpha2|PF08473.10|0.02
>TRINITY_DN42237_c0_g1_i8.p3    type:complete    len:153            (+)     score=-3.34   VGCC_beta4Aa_N|PF12052.7|1
>TRINITY_DN100845_c0_g2_i1.p1   type:complete    len:180            (+)     score=-3.08   sp|Q6VGS5|DAPLE_MOUSE|27.350|5.18e-09
>TRINITY_DN29517_c1_g1_i2.p1    type:complete    len:2177           (-)     score=515.83  sp|Q0VGT4|ZGRF1_MOUSE|46.659|0.0
>TRINITY_DN29517_c1_g1_i5.p1    type:complete    len:2188           (-)     score=515.51  sp|Q0VGT4|ZGRF1_MOUSE|46.659|0.0
>TRINITY_DN29517_c1_g1_i8.p1    type:complete 

Looks like the only search that produced viable results is the "vitellogenin". Will use the top scoring (i.e. best e-value) match for primer design.

#### Create directories

In [10]:
%%bash
mkdir /home/sam/analyses
mkdir /home/sam/analyses/20181129_geoduck_vtg_primers

#### Run Primer3 to design primers

##### Quick explanation:

- Primer3 requires a specially formatted input file. The file must be formatted like this:

```
SEQUENCE_ID=${seq_id}
SEQUENCE_TEMPLATE=${sequence}
PRIMER_TASK=generic
PRIMER_PICK_LEFT_PRIMER=3
PRIMER_PICK_RIGHT_PRIMER=3
PRIMER_OPT_SIZE=18
PRIMER_MIN_SIZE=15
PRIMER_MAX_SIZE=21
PRIMER_MAX_NS_ACCEPTED=1
PRIMER_PRODUCT_SIZE_RANGE=75-150
P3_FILE_FLAG=1
PRIMER_EXPLAIN_FLAG=1
=
```

Values after the "=" on each line can be changed to whatever values the user decides. The ```${sequence}``` must be a nucletoide sequence on a single line, with no line breaks.

The code below uses a ```heredoc``` to write this information to a file. Everything between the following two lines gets printed (via ```cat```) as shown and then redirected to the indicated file (```20181129_primer3_params.txt```):

```
cat << EOF > /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt
EOF
```

- Primer3 is run with the ```--format_output``` to make a nice, human-readable output format.

In [11]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers

# Store sequence only from desired FastA.
# Print all lines after the first line and then delete newlines
# Creates a sequence that exists on a single line, which is necessary for Primer3
sequence=$(tail -n +2 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '\n')

# Store file name of targeted FastA file.
seq_id=$(echo "$(head -n 1 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '>').cds")

# Use heredoc to create Primer3 parameters file
cat << EOF > /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt
SEQUENCE_ID=${seq_id}
SEQUENCE_TEMPLATE=${sequence}
PRIMER_TASK=generic
PRIMER_PICK_LEFT_PRIMER=3
PRIMER_PICK_RIGHT_PRIMER=3
PRIMER_OPT_SIZE=18
PRIMER_MIN_SIZE=15
PRIMER_MAX_SIZE=21
PRIMER_MAX_NS_ACCEPTED=1
PRIMER_PRODUCT_SIZE_RANGE=75-150
P3_FILE_FLAG=1
PRIMER_EXPLAIN_FLAG=1
=
EOF

# Run Primer3
/home/sam/software/primer3-2.4.0/src/primer3_core \
--format_output \
--output=/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers.txt \
/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt

In [12]:
%%bash
cat /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers.txt

PRIMER_ERROR=thermodynamic approach chosen, but path to thermodynamic parameters not specified
=


In [13]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers

# Store sequence only from desired FastA.
# Print all lines after the first line and then delete newlines
sequence=$(tail -n +2 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '\n')

# 
seq_id=$(echo "$(head -n 1 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '>').cds")

# Use heredoc to create Primer3 parameters file
cat << EOF > /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt
SEQUENCE_ID=${seq_id}
SEQUENCE_TEMPLATE=${sequence}
PRIMER_TASK=generic
PRIMER_PICK_LEFT_PRIMER=3
PRIMER_PICK_RIGHT_PRIMER=3
PRIMER_OPT_SIZE=18
PRIMER_MIN_SIZE=15
PRIMER_MAX_SIZE=21
PRIMER_MAX_NS_ACCEPTED=1
PRIMER_PRODUCT_SIZE_RANGE=75-150
P3_FILE_FLAG=1
PRIMER_EXPLAIN_FLAG=1
PRIMER_THERMODYNAMIC_PARAMETERS_PATH=/home/sam/software/primer3-2.4.0/src/primer3_config/
=
EOF

# Run Primer3
/home/sam/software/primer3-2.4.0/src/primer3_core \
--format_output \
--output=/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers.txt \
/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt

In [14]:
%%bash
cat /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers.txt

PRIMER PICKING RESULTS FOR TRINITY_DN51983_c0_g1_i8.p1.cds

No mispriming library specified
Using 0-based sequence positions
OLIGO            start  len      tm     gc%  any_th  3'_th hairpin seq
LEFT PRIMER       1347   18   59.89   55.56    9.11   0.13   42.06 TTACGCCACGGCAACTGT
RIGHT PRIMER      1471   18   60.05   61.11   10.11   0.00    0.00 CGCAGTGCCAACAAGCTG
SEQUENCE SIZE: 14484
INCLUDED REGION SIZE: 14484

PRODUCT SIZE: 125, PAIR ANY_TH COMPL: 10.66, PAIR 3'_TH COMPL: 0.00

    0 ATGGAGCGATTGGTTTTAGCGCTAATCGTTTTGGCGGTCTGTGTCCACGCAGGTCCGATA
                                                                  

   60 CAGAAAATCTCAGTTGATACCTGTGCAAGAACTTGTACAGGCAACAGCAAGTTTAATTAT
                                                                  

  120 GTTGCTGGTAAGACTTACCAGTACAAGTATGATGCCGACATCAAGACTGGGGTCCAGGGA
                                                                  

  180 GCATCAGAAGATAATGCGAAAATACACATGTCGGCAACAGTCGAGATGGAGGTCATATCA
                                         

In [15]:
%%bash
cd /home/sam/analyses/
rsync \
--archive \
--relative \
./20181129_geoduck_vtg_primers/ gannet:/volume1/web/Atumefaciens

#### Run again to generate default output format, just for curiosity

In [16]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers

# Store sequence only from desired FastA.
# Print all lines after the first line and then delete newlines
sequence=$(tail -n +2 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '\n')

# 
seq_id=$(echo "$(head -n 1 /home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds | tr -d '>').cds")

# Use heredoc to create Primer3 parameters file
cat << EOF > /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt
SEQUENCE_ID=${seq_id}
SEQUENCE_TEMPLATE=${sequence}
PRIMER_TASK=generic
PRIMER_PICK_LEFT_PRIMER=3
PRIMER_PICK_RIGHT_PRIMER=3
PRIMER_OPT_SIZE=18
PRIMER_MIN_SIZE=15
PRIMER_MAX_SIZE=21
PRIMER_MAX_NS_ACCEPTED=1
PRIMER_PRODUCT_SIZE_RANGE=75-150
P3_FILE_FLAG=1
PRIMER_EXPLAIN_FLAG=1
PRIMER_THERMODYNAMIC_PARAMETERS_PATH=/home/sam/software/primer3-2.4.0/src/primer3_config/
=
EOF

# Run Primer3
/home/sam/software/primer3-2.4.0/src/primer3_core \
--output=/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers_default_format.txt \
/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_params.txt

cat /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers_default_format.txt

SEQUENCE_ID=TRINITY_DN51983_c0_g1_i8.p1.cds
SEQUENCE_TEMPLATE=ATGGAGCGATTGGTTTTAGCGCTAATCGTTTTGGCGGTCTGTGTCCACGCAGGTCCGATACAGAAAATCTCAGTTGATACCTGTGCAAGAACTTGTACAGGCAACAGCAAGTTTAATTATGTTGCTGGTAAGACTTACCAGTACAAGTATGATGCCGACATCAAGACTGGGGTCCAGGGAGCATCAGAAGATAATGCGAAAATACACATGTCGGCAACAGTCGAGATGGAGGTCATATCAAAGTGTGACCTTGTCATGCGGCTGAATGATGTCACATTGACAGAATATGACCCCATTGACTTGATAATGAAGGAAACGTCATCTGAATTCCGCAGTGGTCTTGAGAAGGCTCCCATCCGAATCTCTTTCCAAGATGGAAACATAGAGGAACTTTGTCTGAAACGAGAGGAGCCAAATTGGGTGCTGAATGCCAAACGTGGAATCCTGTCAATGTTCCAGAACAACATGGACGATTTCTCAACCAATAAAACAGTGTCTGAGACTGATGTGTCTGGTGTGTGTGATACAGAGTACCACGTGAGTAAATCTGGCTGGTACAAGACCACAGTCAGCAAGTCTAAGAATCTCCTAGGTTGTACTGACAGGACCGGGTACAACACTGCCGTCCAGGGAGTGCCATATACCGCAGCTTCTGGCATCCAGTCACTGCCAGTTGTGAAGAGCAACCATAACTGTGAACATGAGATTGACATTAAGTCGCACATTCTCCATTCGGCAACTTGTAATGAACAGCACATGTTCCTGCCATTCTCACGCTCAGACAGTGGAGCTGTAACATACAACACTCAGACTCTCAAGTTCGTGAAAGTAACAACTGGAATCAAGTCACCATTTGATACCAGTACTCGCAAATCAATGAAATTCGATCACATCAATGATGCAGGCAGAGAAGCAAAATCGAGGAAAGATATTACACG

#### Test EMBOSS PrimerSearch on first set of primers picked by Primer3 on source FastA

##### The code below does the following:

- Parses out sequence id, left, and right primers and creates the proper tab-delimited primer sequences file needed by ```primersearch```

- Runs ```primersearch``` using the newly created primer sequences file and the target FastA file that was used to generate our primers in ```Primer3```

In [17]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers/

seq_id=$(grep "SEQUENCE_ID=" /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers_default_format.txt | sed 's/SEQUENCE_ID=//')
left_primer=$(grep "PRIMER_LEFT_0_SEQUENCE=" /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers_default_format.txt | sed 's/PRIMER_LEFT_0_SEQUENCE=//')
right_primer=$(grep "PRIMER_RIGHT_0_SEQUENCE=" /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_primer3_primers_default_format.txt | sed 's/PRIMER_RIGHT_0_SEQUENCE=//')


printf "%s\t" "${seq_id}" "${left_primer}" "${right_primer}" > /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_emboss_primers.txt

# Add newline to end of file
printf "\n" >> /home/sam/analyses/20181129_geoduck_vtg_primers/20181129_emboss_primers.txt

/home/sam/software/EMBOSS-6.6.0/emboss/primersearch \
-auto \
/home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/TRINITY_DN51983_c0_g1_i8.p1.cds \
/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_emboss_primers.txt

cat /home/sam/analyses/20181129_geoduck_vtg_primers/TRINITY_DN51983_c0_g1_i8.primersearch

cat: /home/sam/analyses/20181129_geoduck_vtg_primers/TRINITY_DN51983_c0_g1_i8.primersearch: No such file or directory


In [18]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers/

ls

20181129_emboss_primers.txt
20181129_primer3_params.txt
20181129_primer3_primers_default_format.txt
20181129_primer3_primers.txt
TRINITY_DN51983_c0_g1_i8.p1.cds.for
TRINITY_DN51983_c0_g1_i8.p1.cds.rev
trinity_dn51983_c0_g1_i8.primersearch


In [19]:
%%bash
cat /home/sam/analyses/20181129_geoduck_vtg_primers/trinity_dn51983_c0_g1_i8.primersearch


Primer name TRINITY_DN51983_c0_g1_i8.p1.cds
Amplimer 1
	Sequence: TRINITY_DN51983_c0_g1_i8.p1  
	
	TTACGCCACGGCAACTGT hits forward strand at 1348 with 0 mismatches
	CGCAGTGCCAACAAGCTG hits reverse strand at [13013] with 0 mismatches
	Amplimer length: 125 bp


Primers match up to their source sequence, as expected. Now, to test the primers on the rest of the transcriptome to ensure specificity.

#### Use EMBOSS PrimerSearch tool to test primers across transcriptome

##### The code below does the following:

- Sets variables for file/folder paths

- Runs for loop over all individual CDS FastA files:
    - Uses parameter substitution to strip paths from filenames
    - Uses parameter subsitution to strip extensions from filesnames
    - Uses ```tr``` to convert filenames to lowercase
    - Runs ```primersearch``` on each CDS FastA file
    - Uses ```grep``` to evaluate if the word "Amplimer" is found in the resulting output file; if it is _not_, the file is deleted.

In [20]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers/

fasta_loc="/home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/"
primersearch="/home/sam/software/EMBOSS-6.6.0/emboss/primersearch"
primers="/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_emboss_primers.txt"


time \
for fasta in ${fasta_loc}*.cds
  do
  fasta_no_path=$(echo ${fasta##*/})
  fasta_no_ext=$(echo ${no_path%%.*})
  fasta_no_ext_lower=$(echo ${fasta_no_ext} | tr '[:upper:]' '[:lower:]')
  ${primersearch} -auto ${fasta} ${primers}
  if ! grep --quiet "Amplimer" "${fasta_no_ext_lower}.primersearch"
    then rm ${fasta_no_ext_lower}.primersearch
  fi
done

Process is interrupted.


Killed it to change code to allow primer mismatches.

Also fix extension removal.

In [21]:
%%bash
cd /home/sam/analyses/20181129_geoduck_vtg_primers/

fasta_loc="/home/sam/data/geoduck/transcriptomes/transdecoder_fasta_splits/"
primersearch="/home/sam/software/EMBOSS-6.6.0/emboss/primersearch"
primers="/home/sam/analyses/20181129_geoduck_vtg_primers/20181129_emboss_primers.txt"


time \
for fasta in ${fasta_loc}*.cds
  do
  fasta_no_path=$(echo ${fasta##*/})
  fasta_no_ext=$(echo ${fasta_no_path%%.*})
  fasta_no_ext_lower=$(echo ${fasta_no_ext} | tr '[:upper:]' '[:lower:]')
  ${primersearch} -auto ${fasta} ${primers} 20
  if ! grep --quiet "Amplimer" "${fasta_no_ext_lower}.primersearch"
    then rm ${fasta_no_ext_lower}.primersearch
  fi
done


real	208m42.396s
user	76m45.956s
sys	79m27.744s


In [22]:
%%bash

cd /home/sam/analyses/20181129_geoduck_vtg_primers/

# Check contents of files with matches
for file in *.primersearch
  do
  echo "FILE: ${file}"
  echo ""
  cat ${file}
  echo "----------------------------------"
  echo ""
done

# Copy data to Gannet
cd /home/sam/analyses/
rsync \
--archive \
--relative \
./20181129_geoduck_vtg_primers/ gannet:/volume1/web/Atumefaciens

FILE: trinity_dn51983_c0_g1_i4.primersearch


Primer name TRINITY_DN51983_c0_g1_i8.p1.cds
Amplimer 1
	Sequence: TRINITY_DN51983_c0_g1_i4.p1  
	
	TTACGCCACGGCAACTGT hits forward strand at 1348 with 0 mismatches
	CGCAGTGCCAACAAGCTG hits reverse strand at [12569] with 0 mismatches
	Amplimer length: 125 bp
----------------------------------

FILE: trinity_dn51983_c0_g1_i8.primersearch


Primer name TRINITY_DN51983_c0_g1_i8.p1.cds
Amplimer 1
	Sequence: TRINITY_DN51983_c0_g1_i8.p1  
	
	TTACGCCACGGCAACTGT hits forward strand at 1348 with 0 mismatches
	CGCAGTGCCAACAAGCTG hits reverse strand at [13013] with 0 mismatches
	Amplimer length: 125 bp
----------------------------------



##### OK, only two matches found, both are the same gene, different isoforms. So, I say we're good to go! Will order that primer pair.